In [1]:
import pandas as pd
import logging
import psycopg2

In [2]:
# defining logging level and format
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(name)s - Line# %(lineno)d - '
                                               '%(message)s', datefmt='%d-%b-%y %H:%M:%S')

In [3]:
postgres_host = 'localhost'
postgres_user = 'pawan'
postgres_dbname = 'pawan'
postgres_port = 5432

In [4]:
# establishing connection to postgres database
conn = psycopg2.connect(
    host=postgres_host,
    user=postgres_user,
    port=postgres_port,
    # password=postgres_pwd,
    dbname=postgres_dbname
)

cur = conn.cursor()

In [5]:
# scripts
create_psa_schema = ("""
                     CREATE SCHEMA IF NOT EXISTS psa;
                     COMMIT;
                     """)

create_dm_schema = ("""
                    CREATE SCHEMA IF NOT EXISTS dm;
                    COMMIT;
                    """)

create_company_table = ("""
                        DROP TABLE IF EXISTS psa.company;
                        CREATE TABLE psa.company
                        (
                        company_id      bigint GENERATED ALWAYS AS IDENTITY,
                        company_size    varchar(10),
                        industry        varchar(10)
                        );
                        COMMIT;
                        """)

create_session_table = ("""
                        DROP TABLE IF EXISTS psa.session;
                        CREATE TABLE psa.session
                        (
                        session_id      bigint GENERATED ALWAYS AS IDENTITY,
                        company_id      bigint,  -- added to have link with company
                        created_at      timestamp
                        );
                        COMMIT;
                        """)

create_subscription_table = ("""
                            DROP TABLE IF EXISTS psa.subscription;
                            CREATE TABLE psa.subscription
                            (
                            subscription_id      bigint GENERATED ALWAYS AS IDENTITY,
                            company_id           bigint,
                            subscription_amount  decimal(18,6),
                            created_at           timestamp  -- added to have subcription date
                            );
                            COMMIT;
                            """)

insert_company = ("""
                  -- insering 70% small companies
                  INSERT INTO psa.company
                      (company_size, industry)
                  SELECT 'small' AS company_size,
                          CASE WHEN i % 5 = 0 THEN 'insurance' 
                               WHEN i % 3 = 0 THEN 'banking'
                               WHEN i % 2 = 0 THEN 'telecom'
                               ELSE 'it' 
                              END AS industry  -- assigning industry randomly
                  FROM generate_series(1, 350) s(i);

                  -- insering 30% large companies
                  INSERT INTO psa.company
                      (company_size, industry)
                  SELECT 'large' AS company_size,
                          CASE WHEN i % 5 = 0 THEN 'insurance' 
                               WHEN i % 3 = 0 THEN 'banking'
                               WHEN i % 2 = 0 THEN 'telecom'
                               ELSE 'it' 
                            END AS industry
                  FROM generate_series(1, 150) s(i);
                  COMMIT;
                  """)

insert_sessions = ("""
                    -- inserting sessions for all companies
                    INSERT INTO psa."session"
                        (company_id, created_at)
                    WITH date_series  -- fetch months (feb-dec) of year 2020
                    AS
                    (
                     SELECT DISTINCT DATE_TRUNC('month', generate_series(date '2020-02-01', date '2020-12-31', '1 day')) AS session_date
                    )
                    , company_month  -- cartesian product of company and month date
                    AS
                    (
                     SELECT company_id, session_date,
                            CAST(CASE WHEN company_size = 'small' THEN floor(random()*(6))  -- 1-5 sessions for small company
                                      WHEN company_size = 'large' THEN floor(random()*(11))  -- 1-10 sessions for large company
                                  END AS int) AS rand_num
                     FROM psa.company,
                          date_series
                    )
                    , company_sessions
                    AS
                    (
                     -- random number of session in the first month
                     SELECT company_id,
                            timestamp '2020-01-01' AS session_date
                     FROM (
                           SELECT company_id,
                                  CAST(CASE WHEN company_size = 'small' THEN floor(random()*(6-1)+1)  -- 1-5 sessions for small company
                                            WHEN company_size = 'large' THEN floor(random()*(11-1)+1)  -- 1-10 sessions for large company
                                        END AS int) AS rand_num  -- random number of sessions per company
                           FROM psa.company
                          ) sq,
                          generate_series(1, rand_num) s(i)

                     UNION ALL

                     -- random number of sessions rest of the year
                     SELECT company_id, session_date
                     FROM company_month,
                          generate_series(1, rand_num) s(i)
                    )
                    , company_last_month  -- last active month for each company
                    AS
                    (
                     SELECT company_id, COALESCE(MIN(previous_session_date), date '2020-01-01') AS last_month
                     FROM (
                           SELECT *, 
                                 LAG(session_date, 1) OVER(ORDER BY company_id, session_date) previous_session_date,
                                 LAG(company_id, 1) OVER(ORDER BY company_id, session_date) previous_company_id
                           FROM (
                                 SELECT DISTINCT company_id, session_date
                                 FROM company_sessions
                                ) sq1
                          ) sq2
                     WHERE (previous_company_id IS NULL AND session_date <> date '2020-01-01')
                          OR (previous_company_id = company_id AND (session_date) <> (previous_session_date + interval '1 month'))
                     GROUP BY 1
                    )
                    SELECT cs.company_id,
                           (session_date + trunc(random()  * 27) * '1 day'::interval) + random() * 24 * interval '1 hour' AS created_at
                    FROM company_sessions cs
                    LEFT JOIN company_last_month cm
                        ON cs.company_id = cm.company_id
                    WHERE cs.session_date <= cm.last_month OR cm.last_month IS NULL  -- discarding sessions after the churn
                    ORDER BY 2, 1;
                    COMMIT;
                    """)

insert_subscription = ("""
                       -- inserting subscription data
                       INSERT INTO psa."subscription"
                           (company_id, subscription_amount, created_at)
                       SELECT DISTINCT s.company_id,
                              CASE WHEN c.company_size = 'small' THEN 19
                                   WHEN c.company_size  = 'large' THEN 99
                                 END AS subscription_amount,
                              DATE_TRUNC('month', s.created_at) AS created_at
                       FROM psa."session" s
                       LEFT JOIN psa.company c 
                           ON s.company_id = c.company_id
                       ORDER BY 3, 1;
                       COMMIT;
                       """)

In [6]:
script_dict = {'create PSA schema': create_psa_schema, 
               'create DM schema': create_dm_schema,
               'create company table': create_company_table,
               'create session table': create_session_table,
               'create subscription table': create_subscription_table,
               'insert company data': insert_company,
               'insert sessions data': insert_sessions,
               'insert subscription data': insert_subscription
              }

In [7]:
for txt, script in script_dict.items():
    logging.info(f"Executing script to {txt}")
    cur.execute(script)

21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to create PSA schema
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to create DM schema
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to create company table
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to create session table
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to create subscription table
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to insert company data
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to insert sessions data
21-Dec-20 23:14:06 - INFO - root - Line# 2 - Executing script to insert subscription data


In [8]:
pd.read_sql('SELECT * FROM psa.company LIMIT 20', conn)

,company_id,company_size,industry
0,1,small,it
1,2,small,telecom
2,3,small,banking
3,4,small,telecom
4,5,small,insurance
5,6,small,banking
6,7,small,it
7,8,small,telecom
8,9,small,banking
9,10,small,insurance


In [9]:
pd.read_sql('SELECT * FROM psa.session LIMIT 20', conn)

,session_id,company_id,created_at
0,1,360,2020-01-01 01:12:04.299794
1,2,359,2020-01-01 01:14:13.982629
2,3,461,2020-01-01 01:51:31.135858
3,4,404,2020-01-01 02:14:23.809519
4,5,446,2020-01-01 02:19:13.540302
5,6,278,2020-01-01 02:32:07.634217
6,7,277,2020-01-01 03:23:38.272735
7,8,7,2020-01-01 03:50:29.678459
8,9,15,2020-01-01 03:54:55.063974
9,10,128,2020-01-01 04:16:22.056787


In [10]:
pd.read_sql('SELECT * FROM psa.subscription LIMIT 20', conn)

,subscription_id,company_id,subscription_amount,created_at
0,1,1,19.0,2020-01-01
1,2,2,19.0,2020-01-01
2,3,3,19.0,2020-01-01
3,4,4,19.0,2020-01-01
4,5,5,19.0,2020-01-01
5,6,6,19.0,2020-01-01
6,7,7,19.0,2020-01-01
7,8,8,19.0,2020-01-01
8,9,9,19.0,2020-01-01
9,10,10,19.0,2020-01-01


In [11]:
# scripts for metrics
metric_1 = ("""
            SELECT created_at AS report_month,
                    SUM(subscription_amount)::INT AS revenue
            FROM psa."subscription"
            GROUP BY 1
            ORDER BY 1;
            """)

metric_2 = ("""
            SELECT s.created_at AS report_month,
                    COUNT(s.company_id) AS active_customers,
                    SUM(CASE WHEN c.company_size = 'small' THEN 1 ELSE 0 END) AS active_small_customers,
                    SUM(CASE WHEN c.company_size = 'large' THEN 1 ELSE 0 END) AS active_large_customers
            FROM psa."subscription" s
            INNER JOIN psa.company c
                ON s.company_id = c.company_id
            GROUP BY 1
            ORDER BY 1;
            """)

metric_3 = ("""
            SELECT DATE_TRUNC('month', created_at) AS report_month,
                    COUNT(1) AS sessions
            FROM psa."session" s
            GROUP BY 1
            ORDER BY 1;
            """)

In [12]:
pd.read_sql(metric_1, conn)

,report_month,revenue
0,2020-01-01,21500
1,2020-02-01,18685
2,2020-03-01,16756
3,2020-04-01,14964
4,2020-05-01,13248
5,2020-06-01,11528
6,2020-07-01,10322
7,2020-08-01,9325
8,2020-09-01,8621
9,2020-10-01,7601


In [13]:
pd.read_sql(metric_2, conn)

,report_month,active_customers,active_small_customers,active_large_customers
0,2020-01-01,500,350,150
1,2020-02-01,415,280,135
2,2020-03-01,364,241,123
3,2020-04-01,316,204,112
4,2020-05-01,272,171,101
5,2020-06-01,232,143,89
6,2020-07-01,198,116,82
7,2020-08-01,175,100,75
8,2020-09-01,159,89,70
9,2020-10-01,139,77,62


In [14]:
pd.read_sql(metric_3, conn)

,report_month,sessions
0,2020-01-01,1848
1,2020-02-01,1558
2,2020-03-01,1412
3,2020-04-01,1222
4,2020-05-01,1010
5,2020-06-01,884
6,2020-07-01,811
7,2020-08-01,717
8,2020-09-01,654
9,2020-10-01,591


In [15]:
conn.commit()
conn.close()